In [13]:
from pycox.evaluation import EvalSurv
import torch
import torchtuples as tt
import os
import pickle
from lifelines.utils import concordance_index

from utils import dataset_name
from eval_utils import MODEL_CLASS, get_results, DL_MODELS, ML_MODELS, MODELS
import numpy as np
import pandas as pd
np.random.seed(42)
_ = torch.manual_seed(42)


print(f"Experiments over {dataset_name}")

Experiments over NWTCO


In [14]:
train = pd.read_csv(f"../../datasets/train/{dataset_name}.csv")
test = pd.read_csv(f"../../datasets/test/{dataset_name}.csv")

X, y = train.drop(["event", "duration"], axis=1), train[["event", "duration"]]
X_test, y_test = test.drop(["event", "duration"], axis=1), test[["event", "duration"]]

# Finding the best parameters for each model and run the models over the test data

In [15]:
results = pd.DataFrame()
     
for model in MODELS:
    stats = pickle.load( open( f"statistics/{model}/best_model.pkl", "rb" ))
    df = pd.DataFrame(stats, index=[model])
    results = pd.concat([results, df], axis=0)
    
c_index_df = results[["c_index", "c_index_std", "c_index_params"]].sort_values("c_index", ascending=False)
concordance_td_df = results[["concordance_td", "concordance_td_std", "concordance_td_params"]].sort_values("concordance_td", ascending=False)
ibs_df = results[["ibs", "ibs_std", "ibs_params"]].sort_values("ibs", ascending=True)

## `c index`

In [16]:
c_index_df

,c_index,c_index_std,c_index_params
deep_surv,0.730760,0.022420,"deep_surv_dropout_0.1_num_nodes_[28, 28, 100, ..."
rsf,0.724071,0.022968,rsf_n_estimators_100_max_depth_5_min_samples_s...
gbst,0.723353,0.020226,gbst_learning_rate_0.1_n_estimators_100_subsam...
cox_time,0.721125,0.024682,"cox_time_dropout_0.1_num_nodes_[32, 32]_activa..."
reg_coxph,0.713272,0.018887,reg_coxph_l1_ratio_0.8_tol_1e-07_max_iter_1000...
pc_hazard,0.604035,0.070321,"pc_hazard_dropout_0.1_num_nodes_[16, 16, 16, 1..."
deep_hit,0.461041,0.082896,"deep_hit_dropout_0.3_num_nodes_[3, 5]_activati..."


In [17]:
results = []
column = "c_index_params"


for model_name, params in zip(c_index_df.index, c_index_df[column]):
    if model_name in DL_MODELS:
        c_index, concordance_td, ibs = get_results(params, model_name, X, y, X_test, y_test)
        results.append((model_name, c_index))
    else:
        c_index, concordance_td, ibs = get_results(params, model_name, X, y, X_test, y_test)
        results.append((model_name, c_index))

for m,val in sorted(results, key=lambda x: x[1], reverse=True):
    print(m, np.round(val,4))

gbst 0.6993
deep_surv 0.6943
rsf 0.6869
cox_time 0.6851
reg_coxph 0.6825
deep_hit 0.5988
pc_hazard 0.5135


___

## `concordance td`

In [6]:
concordance_td_df

,concordance_td,concordance_td_std,concordance_td_params
deep_surv,0.728625,0.022081,"deep_surv_dropout_0.1_num_nodes_[28, 28, 100, ..."
deep_hit,0.723635,0.023275,"deep_hit_dropout_0.3_num_nodes_[28, 28, 100, 2..."
rsf,0.720648,0.021982,rsf_n_estimators_200_max_depth_3_min_samples_s...
gbst,0.720532,0.022543,gbst_learning_rate_0.001_n_estimators_100_subs...
cox_time,0.718467,0.021232,"cox_time_dropout_0.1_num_nodes_[32, 32]_activa..."
reg_coxph,0.711224,0.021178,reg_coxph_l1_ratio_0.8_tol_1e-07_max_iter_1000...
pc_hazard,0.461036,0.066815,"pc_hazard_dropout_0.1_num_nodes_[16, 16, 16, 1..."


In [18]:
results = []
column = "concordance_td_params"


for model_name, params in zip(concordance_td_df.index, concordance_td_df[column]):
    if model_name in DL_MODELS:
        c_index, concordance_td, ibs = get_results(params, model_name, X, y, X_test, y_test)
        results.append((model_name, concordance_td))
    else:
        c_index, concordance_td, ibs = get_results(params, model_name, X, y, X_test, y_test)
        results.append((model_name, concordance_td))

for m,val in sorted(results, key=lambda x: x[1], reverse=True):
    print(m, np.round(val,4))

deep_hit 0.7135
rsf 0.7022
gbst 0.6953
deep_surv 0.694
cox_time 0.6856
reg_coxph 0.6823
pc_hazard 0.395


## `integrated brier score`

In [8]:
ibs_df

,ibs,ibs_std,ibs_params
pc_hazard,0.019188,0.004434,"pc_hazard_dropout_0.1_num_nodes_[16, 16, 16, 6..."
deep_surv,0.104011,0.009863,"deep_surv_dropout_0.1_num_nodes_[16, 32, 64, 6..."
rsf,0.104069,0.009621,rsf_n_estimators_100_max_depth_5_min_samples_s...
reg_coxph,0.106276,0.009826,reg_coxph_l1_ratio_0.9_tol_1e-09_max_iter_1000...
gbst,0.106295,0.009731,gbst_learning_rate_0.1_n_estimators_100_subsam...
cox_time,0.108626,0.008861,"cox_time_dropout_0.2_num_nodes_[32, 32]_activa..."
deep_hit,0.324318,0.076551,"deep_hit_dropout_0.3_num_nodes_[3, 5]_activati..."


In [19]:
results = []
column = "ibs_params"


for model_name, params in zip(ibs_df.index, ibs_df[column]):
    if model_name in DL_MODELS:
        c_index, concordance_td, ibs = get_results(params, model_name, X, y, X_test, y_test)
        results.append((model_name, ibs))
    else:
        c_index, concordance_td, ibs = get_results(params, model_name, X, y, X_test, y_test)
        results.append((model_name, ibs))

for m,val in sorted(results, key=lambda x: x[1], reverse=False):
    print(m, np.round(val,4))

gbst 0.1101
rsf 0.113
deep_surv 0.1143
reg_coxph 0.1186
cox_time 0.1216
deep_hit 0.5128
pc_hazard 0.8398
